# HW4: Hidden Markov Model (Loaded or Fair Die?)

In [106]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sla
from logdouble import logdouble

tol = 1e-6

1.) Pre-process the `rolls.txt` file into a vector

In [107]:
sides = 6
num_rolls = 3000
rolls = np.zeros(num_rolls, dtype=int)
counter = 0
with open("rolls.txt") as file:
    for line in file:
        roll = int(line)
        rolls[counter] = roll
        counter += 1
rolls = np.array(rolls)

2.) Initialize our transition, emission, and initial probabilities: `A` will be the transition matrix (with shape $2,2$), `B` will be the emission matrix (with shape $2, 6$), and `pi` will be our initial probability vector of length $2$. From here on, die `0` refers to the fair die, while die `1` refers to the loaded die. I'm using this terminology and [this Wikipedia article](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm), as I think the steps explained there are a bit clearer than the notes.

In [108]:
A = np.ndarray((2, 2), dtype=float)
#Let's call the 1-index the loaded die, and the 0-index the fair die
A[0][0] = 0.9
A[0][1] = 0.1
A[1][0] = 0.2
A[1][1] = 0.8

#Given we're using dice i, whats the probability we roll j?
B = np.ndarray((2, 6), dtype=float)
B[0] = 1/sides
B[1] = 1/sides

#Guess for initial die
pi = np.array([0.5, 0.5])

3.) Define our parameter updates function

In [109]:
#UPDATE PARAMETERS FUNCTION
def update_parameters(gamma, xi, T):
    """
    returns pi*, A*, B*
    """
    pi_star = np.array([gamma[0][0], gamma[0][1]])
    numerator_matrix = np.zeros((2, 2))
    denominator_matrix = np.zeros((2, 2))
    for t in range(T - 1):
        for i in range(2):
            denominator_matrix[i][0] += gamma[t][i]
            denominator_matrix[i][1] = denominator_matrix[i][0]
            for j in range(2):
                numerator_matrix[i][j] += xi[t][i][j]
    A_star = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            A_star[i][j] = numerator_matrix[i][j]/denominator_matrix[i][j]
    
    
    numerator_matrix = np.zeros((2, 6))
    denominator_matrix = np.zeros((2, 6))
    for t in range(T):
        for i in range(2):
            for j in range(6):
                denominator_matrix[i][j] += gamma[t][i]

            for j in range(6):
                if rolls[t] - j == 0:
                    numerator_matrix[i][j] += gamma[t][i]

    B_star = np.zeros((2, 6))
    for i in range(2):
        for j in range(6):
            B_star[i][j] = numerator_matrix[i][j]/denominator_matrix[i][j]

    return pi_star, A_star, B_star

4.) Initialize `alpha`, `beta`, `gamma`, and `xi`:

In [110]:
T = 300
alpha = np.zeros((T, 2))
beta = np.zeros((T, 2))
gamma = np.zeros((T, 2))
xi = np.zeros((T, 2, 2))

5.) Execute Baum-Welch Algorithm. Going to scale `alpha` and `beta` to ensure we avoid underflow. I found [these slides](https://courses.engr.illinois.edu/ece417/fa2021/lectures/lec16.pdf) that show how we can mitigate this problem.

In [113]:
alpha_hat = beta_hat = np.zeros((T, 2))
alpha_tilde = beta_tilde = np.zeros((T, 2))
#g is our scaling constant vector
g = np.zeros(T)
g[0] = 1 
while True:
    #ALPHAs:
    #initialize alphas
    
    alpha[0][0] = pi[0]*B[0][rolls[0]]
    alpha[0][1] = pi[1]*B[1][rolls[0]]
    #recursively compute for each timestep
    for t in range(0, T - 1):
        for i in range(2):
            alpha[t + 1][i] = B[i][rolls[t + 1]]
            sum = 0.0
            for j in range(2):
                sum += alpha[t][j]*A[j][i]
            alpha[t + 1][i] *= sum
    break
            

In [111]:
while True:
    #ALPHAs:
    #initialize alphas
    alpha[0][0] = pi[0]*B[0][rolls[0]]
    alpha[0][1] = pi[1]*B[1][rolls[0]]
    #recursively compute for each timestep
    for t in range(0, T - 1):
        for i in range(2):
            alpha[t + 1][i] = B[i][rolls[t + 1]]
            sum = 0.0
            for j in range(2):
                sum += alpha[t][j]*A[j][i]
            alpha[t + 1][i] *= sum

    #BETAS:
    #initialize betas
    beta[T - 1][0] = beta[T - 1][1] = 1
    for t in reversed(range(T - 1)):
        for i in range(2):
            for j in range(2):
                beta[t][i] += beta[t + 1][j]*A[i][j]*B[j][rolls[t + 1]]
    
    #GAMMAS:
    for t in range(T):
        # i = 0
        numerator_0 = alpha[t][0]*beta[t][0]
        numerator_1 = alpha[t][1]*beta[t][1]
        denominator = numerator_0 + numerator_1
        if denominator == 0:
            gamma[t][0] = gamma[t][1] = 0
            continue
        gamma[t][0] = numerator_0/denominator
        gamma[t][1] = 1 - gamma[t][0]


    #XIS
    for t in range(T- 1):
        #only need to calculate denominator once per time-step
        denom = 0.0
        for k in range(2):
            for w in range(2):
                term = alpha[t][k]*A[k][w]*beta[t + 1][w]*B[w][rolls[t + 1]]
                denom += term

        for i in range(2):
            for j in range(2):
                num = alpha[t][i]*A[i][j]*beta[t+1][j]*B[j][rolls[t + 1]]
                xi[t][i][j] = num/denom

    #Now update parameters            
    pi_star, A_star, B_star = update_parameters(gamma, xi, T)
    pi_norm = sla.norm(pi_star - pi)
    A_norm = sla.norm(A_star - A)
    B_norm = sla.norm(B_star - B)

    #Terminate once updates are within tolerance of last iteration
    if pi_norm < tol and A_norm < tol and B_norm < tol:
        break
    else:
        pi = pi_star
        A = A_star
        B = B_star                

6.) Now, we can view our final transmission, emission, and initial probabilities:

In [112]:
A_final = A
B_final = B
pi_final = pi

print(f"Transition Matrix:\n{A_final}\n\n")
print(f"Emission Matrix:\n{B_final}\n\n")
print(f"Initial probabilities:\n{pi_final}\n\n")

Transition Matrix:
[[0.83348245 0.16649693]
 [0.29550444 0.70453135]]


Emission Matrix:
[[9.08239827e-06 2.01843272e-01 1.97877371e-01 1.36712376e-01
  1.36690939e-01 3.26866960e-01]
 [3.46686944e-01 6.90654216e-02 4.85834270e-02 1.91329798e-01
  1.91367038e-01 1.52967371e-01]]


Initial probabilities:
[9.40989105e-24 1.00000000e+00]


